# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Part 2 of the assigment

#### Importing Libraries

In [1]:
import numpy as np #Numpy Library
import pandas as pd #Panda Library

In [2]:
from bs4 import BeautifulSoup #BeautifulSoup to read the html

In [3]:
import requests #to request the page

In [4]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [ ]:
!conda install -c conda-forge geocoder
import geocoder

Fetching package metadata .............
Solving package specifications: 

#### PART 1 of the assigment

In [ ]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [ ]:
soup = BeautifulSoup(source,'lxml')

In [ ]:
tabla = soup.find('table', attrs={'class':'wikitable sortable'})
#print(tabla.prettify())  #prettify to display in a more friendly way

In [ ]:
filas = tabla.find_all('tr')
l=[]
for tr in filas:
    td = tr.find_all('td')
    fila = [tr.text for tr in td]
    l.append(fila)
lista = pd.DataFrame(l, columns=['PostCode','Borough','Neigborhood'])

In [ ]:
lista = lista.replace('\n','', regex=True)

In [ ]:
lista.drop(lista.index[0],inplace=True)

In [ ]:
lista = lista[lista.Borough != 'Not assigned']

In [ ]:
lista['Neigborhood']= np.where(lista['Neigborhood']=='Not assigned',lista['Borough'],lista['Neigborhood'])

In [ ]:
lista.sort_values('PostCode',inplace=True)

In [ ]:
concatenar = {
    'Neigborhood': lambda x: ', '.join(x)
}

In [ ]:
lista = lista.groupby(['PostCode','Borough'], as_index=False, sort=False).agg(concatenar)

## Part 2 of the assigment

Reading the csv file with the geospatial data for the postal codes that the geocoder returns error

In [ ]:
gd=pd.read_csv('https://cocl.us/Geospatial_data',index_col=0)
print('Data loaded')

In [ ]:
gd.head()

In [ ]:
# The code was removed by Watson Studio for sharing.

Using the geocoder to obtain the latitude and longitude of the Postal Code

In [ ]:
lista['Latitude']=0
lista['Longitude']=0
for i in range(0, len(lista)):
    ubicacion ='\''+lista['Borough'].iloc[i]+','+lista['PostCode'].iloc[i]+'\''
    try:
        g=geocoder.here(ubicacion,app_id=ID,app_code=CODE) # I'm using the geocoder HERE
        #print(i,"-",ubicacion,':',g.lat,' ',g.lng)
        lista['Latitude'].iloc[i] = g.lat
        lista['Longitude'].iloc[i]= g.lng
    except: #If the geocoder returns an error, I use gd dataframe to look for the latitude and longitude of the postal code
        postalcode=lista['PostCode'].iloc[i]
        lista['Latitude'].iloc[i] = gd['Latitude'].loc[postalcode]
        lista['Longitude'].iloc[i]= gd['Longitude'].loc[postalcode]
        #print(i,"-",ubicacion,':', lista['Latitude'].iloc[i],' ',lista['Longitude'].iloc[i])

In [ ]:
lista.head(50)

In [ ]:
lista.shape

In [ ]:
lista.info()

In [ ]:
lista.tail(53)